<a href="https://colab.research.google.com/github/smwaingeni-ai/AgriX-AfricaDeepTech2025/blob/main/notebooks/crop_disease_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**🔹 Step 1: Install Required Libraries**

In [1]:
!pip install tensorflow matplotlib

**Step 2: Import Libraries**

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


**step 3: Clone Your GitHub Repo**

In [7]:
# Remove old folder (optional)
!rm -rf AgriX-AfricaDeepTech2025

# Clone again
!git clone https://github.com/smwaingeni-ai/AgriX-AfricaDeepTech2025.git

# Change directory
%cd AgriX-AfricaDeepTech2025


Cloning into 'AgriX-AfricaDeepTech2025'...
remote: Enumerating objects: 596, done.
remote: Counting objects: 100% (325/325), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 596 (delta 190), reused 81 (delta 81), pack-reused 271 (from 2)
Receiving objects: 100% (596/596), 24.39 MiB | 14.16 MiB/s, done.
Resolving deltas: 100% (229/229), done.
/content/AgriX-AfricaDeepTech2025


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
data_dir = "/content/AgriX-AfricaDeepTech2025/data/crops/plantvillage_subset"
img_size = (128, 128)
batch_size = 16

# Generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

print("✅ Detected classes:", list(train_generator.class_indices.keys()))


Found 35 images belonging to 3 classes.
Found 8 images belonging to 3 classes.
✅ Detected classes: ['Healthy', 'Maize___Leaf_Spot', 'Tomato___Bacterial_spot']


**Step 2: Build the CNN Model**

In [9]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

**Step 3: Train the Model**

In [10]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.2303 - loss: 1.4289

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 520ms/step - accuracy: 0.2437 - loss: 1.4729 - val_accuracy: 0.5000 - val_loss: 1.2066
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 432ms/step - accuracy: 0.5139 - loss: 1.1154 - val_accuracy: 0.3750 - val_loss: 1.1386
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 513ms/step - accuracy: 0.3810 - loss: 1.0884 - val_accuracy: 0.3750 - val_loss: 1.0550
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 589ms/step - accuracy: 0.3492 - loss: 1.0406 - val_accuracy: 0.7500 - val_loss: 0.9896
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 400ms/step - accuracy: 0.6557 - loss: 0.9487 - val_accuracy: 0.7500 - val_loss: 0.8424


**Step 4: Export to TFLite for Mobile Use**

In [11]:
# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save model
with open("crop_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved!")


Saved artifact at '/tmp/tmpa19pgsoh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140060670751568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140063349912400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670754448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670746384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670752144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670751760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670750416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670752720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670748880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060670749840: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ TFLite model s

In [22]:
import tensorflow as tf
import os

# ✅ Get class names
class_names = list(train_generator.class_indices.keys())

# === 1. Define and train a simple CNN model ===
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=5)

# === 2. Convert model to TFLite ===
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# === 3. Save the TFLite model ===
output_path = "/content/AgriX-AfricaDeepTech2025/prototype/tflite_model/crop_disease_model.tflite"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite model saved to: {output_path}")

# === 4. Verify ===
!ls -lh /content/AgriX-AfricaDeepTech2025/prototype/tflite_model/


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - accuracy: 0.3612 - loss: 1.8212
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 0.3641 - loss: 1.1686
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 455ms/step - accuracy: 0.6520 - loss: 0.8655
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 308ms/step - accuracy: 0.7277 - loss: 0.8472
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 415ms/step - accuracy: 0.5995 - loss: 0.8321
Saved artifact at '/tmp/tmpt_q1q74r'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_41')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140060528942480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060528941520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060528940944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060528939792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140060528941136: TensorSpe